# Introduction to Quantum Computing #

## MITP 2023

The aim of this notebook is to introduce the reader to using quantum devices, specifically superconducting qubit devices such as those being developed and produced by IBM. Most of the code will be written in Qiskit, with brief introductions to other quantum computing languages which will needed for the rest of the course. 

# Qiskit

In this section we will introduce the Qiskit language, manipulating single and multiqubit states using the set of [universal quantum gates](https://qiskit.org/documentation/tutorials/circuits/3_summary_of_quantum_operations.html). The tutorial will lead up to the SWAP test algorithm, which is prolific in quantum machine learning.

In [ ]:
### Import required packages
import numpy as np
import matplotlib.pyplot as plt
from qiskit import (QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute)
from qiskit.quantum_info import Statevector
from qiskit.tools.monitor import job_monitor
from qiskit.visualization import *

## The qubit

The building block of superconducting quantum devices is the so called qubit, the quantum analogue of the classical bit. The state of the qubit can be modelled as point on the surface of a three-dimensional Bloch sphere, such that a state $\vert \psi\rangle$ can be written as 

$$
\vert \psi \rangle = \cos \frac{\theta}{2} \vert 0 \rangle + e^{i\phi}\sin \frac{\theta}{2} \vert 1 \rangle,
$$

where $0\leq\theta<\pi$ and $0\leq\phi<2\pi$ corrspond to angles on the Bloch sphere. The state can then be manipulated by rotating the vector using the rotation

$$
U_3 (\theta, \phi, \lambda) = \begin{pmatrix} \cos \big( \frac{\theta}{2} \big) & - e^{i\lambda} \sin \big( \frac{\theta}{2} \big) \\ e^{i\phi} \sin \big( \frac{\theta}{2} \big) & e^{i(\phi + \lambda)} \cos \big( \frac{\theta}{2} \big) \end{pmatrix},
$$

where $e^{i\lambda}$ is a phase which has observable effect, as only the relative phase between the two basis states $\vert0\rangle$ and $\vert1\rangle$ has a physical meaning. 

The $U_3$ rotation is realised in Qiskit using the `QuantumCircuit.u(\theta, \phi, \lambda, qubit)` gate operation, demonstrated below:

1) The qubit is initialised in the $\vert 0 \rangle$ state
2) The $U_3$ rotation is applied with parameters $\theta = \pi/2$, $\phi = \pi/4$ and $\lambda = 0$

In [ ]:
#Initilise quantum circuit object with 1 qubit and 1 classical readout bit
circuit = QuantumCircuit(1, 1)
#Apply U_3 roation
circuit.u(np.pi/2, np.pi/4, 0, 0)
#Draw circuit
circuit.draw('mpl')

We can now plot the qubit as a Bloch shere using the `StateVector` method, which provides the statevector of each qubit in the quantum circuit. Here, we only have one qubit and so one Bloch sphere, but we can plot using `plot_bloch_multivector(state)` which will plot a Bloch sphere diagram for all qubits in the circuit.

In [ ]:
#Get state vector from the circuit
state = Statevector(circuit)
#Plot Bloch sphere
plot_bloch_multivector(state)

We now want to perform a measurement on the qubit and run the circuit on a quantum device. The device we will use here is the `qasm_simulator` available through Aer (runs locally). 

In [ ]:
#Measure qubit to classical bit
circuit.measure(0, 0)

circuit.draw('mpl')

In [ ]:
#Load the backend we want
sim = Aer.get_backend('qasm_simulator')
#Run the circuit on the quantum simulator from Aer (runs locally)
#for 1000 shots on the device
job = execute(circuit, sim, shots=1000)
job_monitor(job) #monitors the job's progression (useful when running on IBMQ)
#Retrieve the number of counts for each state measured
counts = job.result().get_counts()
print(counts)

We now have a dictionary of counts against measured states (the states are classical bit strings). Here we have used a projective measurement 

$$
\vert{\psi}\rangle \hspace{5pt} \longrightarrow{M_m} \hspace{5pt} \vert{\psi^\prime}\rangle = \frac{M_m\vert{\psi}\rangle}{\sqrt{\langle{\psi}\vert M^\dagger_m M_m \vert{\psi}\rangle}},
$$

and thus we must run the circuit many times to build a probability distribution. The `counts` dictionary now contains the information about how many times the system was measured in each possible configuration. We can visualise these counts using the `plot_histogram` method from Qiskit.

In [ ]:
plot_histogram(counts)

## Multiple qubits

The state space of a quantum computer grows exponentially with the number of qubits, such that an $n$-qubit system has $2^n$ basis states in the computational basis. These basis states take the form $\vert{q_1q_2,\dots,q_n}\rangle$, where $q_i\in\{0,1\}$.

As a first example, we will construct a Bell state on 2 qubits,

$$
\frac{1}{\sqrt{2}} \big(\vert{00}\rangle + \vert{11}\rangle \big),
$$

and perform a measurement on the qubits. Here we will use the `QuantumRegister` and `ClassicalRegister` objects, which will be useful when we get to more complicated algorithms. The registers are a collection of (qu)bits which we can pass to the QuantumCircuit operation. It makes our life easier when we have many qubits!  

In [ ]:
q  = QuantumRegister(2, name='q')
qc = ClassicalRegister(2, name='qc')

bell_state = QuantumCircuit(q, qc)

bell_state.h(q[0])
bell_state.cx(q[0],q[1])

bell_state.measure(q, qc)

bell_state.draw('mpl')

In [ ]:
job = execute(bell_state, sim, shots=1000)
job_monitor(job)
counts = job.result().get_counts()
plot_histogram(counts)

The circuit returns the expected measurement of a Bell state. Let us know run this on a real device to see how noise will affect the result. 

Here we will use the IBM Quantum Experience. If you haven't signed up for an account yet, please skip the next cell!

In [ ]:
from qiskit_ibm_provider import IBMProvider

#IBMProvider.save_account('APIKEY', overwrite=True)
provider = IBMProvider.load_account()
device   = provider.get_backend('ibmq_quito')

job = execute(bell_state, device, shots=1000)
job_monitor(job)
qcounts = job.result().get_counts()
plot_histogram([counts, qcounts], legend=['Sim', 'Quito'])

## Exercise : Build an arbitrary state

Can you write a circuit which builds the following state on two qubits?

$$
\vert \psi \rangle = \frac{1}{\sqrt{3}} \big( \vert 00 \rangle + \vert 10 \rangle + \vert 01 \rangle \big)
$$

Can you do this with minimal circuit depth?

In [ ]:
### Solution 

circuit = QuantumCircuit(2)

circuit.u(2*np.arccos(1/np.sqrt(3)), 0, 0, 0)
circuit.ch(0,1)
circuit.cx(1,0)

circuit.measure_all()

circuit.draw('mpl')

In [ ]:
job = execute(circuit, sim, shots=1000)
job_monitor(job)
counts = job.result().get_counts()
plot_histogram(counts)

## The SWAP Test

First introduced in Refs [1] and [2], the SWAP test is a small algorithm which determines how close two quantum states $\vert \phi \rangle$ and $\vert \psi \rangle$ are. The test is very common in quantum machine learning.

The circuit is very simple and consists of three operations on three qubits: the ancilla qubit $a$, the $\phi$ qubit in the $\vert \phi \rangle$ state, and the $\psi$ qubit in the $\vert \psi \rangle$ state.

In [ ]:
phi = QuantumRegister(1, name='p')
psi = QuantumRegister(1, name='q')
a   = QuantumRegister(1, name='a')

swapTest = QuantumCircuit(a, phi, psi)

swapTest.h(a)
swapTest.cswap(a, phi, psi)
swapTest.h(a)

swapTest.draw('mpl')

The first Hadamard operaton puts the ancilla qubit $a$ into a linear superposition of the computation basis

$$
\vert 0 \rangle \otimes \vert \phi \rangle \otimes \vert \psi \rangle \xrightarrow{H} \frac{1}{\sqrt{2}} \big(\vert 0 \rangle + \vert 1 \rangle\big) \otimes \vert \phi \rangle \otimes \vert \psi \rangle.
$$

The controlled-SWAP gate is then applied across all three qubits, controlled from the ancilla. If the ancilla is in the $\vert 1 \rangle$ state, the SWAP operation is applied to the $p$ and $q$ qubits, swapping the states of the qubits.

$$
\frac{1}{\sqrt{2}} \big(\vert 0 \rangle + \vert 1 \rangle\big) \otimes \vert \phi \rangle \otimes \vert \psi \rangle \xrightarrow{cSWAP} \frac{1}{\sqrt{2}} \vert 0 \rangle \otimes \vert \phi \rangle \otimes \vert \psi \rangle + \frac{1}{\sqrt{2}} \vert 1 \rangle \otimes \vert \psi \rangle \otimes \vert \phi \rangle.
$$

A second Hadamard gate is then applied to the ancilla qubit

$$
\frac{1}{\sqrt{2}} \vert 0 \rangle \otimes \vert \phi \rangle \otimes \vert \psi \rangle + \frac{1}{\sqrt{2}} \vert 1 \rangle \otimes \vert \psi \rangle \otimes \vert \phi \rangle \xrightarrow{H} \frac{1}{2} \Bigg( \big(\vert 0 \rangle + \vert 1 \rangle \big) \otimes \vert \phi \rangle \otimes \vert \psi \rangle + \big(\vert 0 \rangle - \vert 1 \rangle \big) \otimes \vert \psi \rangle \otimes \vert \phi \rangle \Bigg). 
$$

Therefore, we can see that if $\vert \phi \rangle = \vert \psi \rangle$ then a measurement of the ancilla will yield the $\vert 0 \rangle$ with certainty. If the states are orthogonal, then one will measure an equal superposition of $\vert 0 \rangle$ and $\vert 1 \rangle$ on the ancilla. 

In [ ]:
ac = ClassicalRegister(1, name='ac')
swapTest = QuantumCircuit(a, phi, psi, ac)

swapTest.h(a)
swapTest.cswap(a, phi, psi)
swapTest.h(a)

swapTest.measure(a, ac)
swapTest.draw('mpl')

In [ ]:
job = execute(swapTest, sim, shots=1000)
job_monitor(job)
counts = job.result().get_counts()
plot_histogram(counts)

In [ ]:
ac = ClassicalRegister(1, name='ac')
swapTest = QuantumCircuit(a, phi, psi, ac)

swapTest.x(phi)

swapTest.h(a)
swapTest.cswap(a, phi, psi)
swapTest.h(a)

swapTest.measure(a, ac)
swapTest.draw('mpl')

In [ ]:
job = execute(swapTest, sim, shots=1000)
job_monitor(job)
counts = job.result().get_counts()
plot_histogram(counts)

## Calculating an Inner Product with the SWAP test

Now, let us consider the scenario where we want to calculate the inner product of the states $\vert \phi \rangle$ and $\vert \psi \rangle$, we can do this using the SWAP test. Rewriting the final state from above, we can see that the final state is

$$
\frac{1}{2}\vert 0 \rangle \otimes \big( \vert \phi \rangle \otimes \vert \psi \rangle + \vert \psi \rangle \otimes \vert \phi \rangle \big) + \frac{1}{2}\vert 1 \rangle \otimes \big( \vert \phi \rangle \otimes \vert \psi \rangle - \vert \psi \rangle \otimes \vert \phi \rangle\big).
$$

The probability of measuring the $\vert 0 \rangle$ on the ancilla qubit is therefore

$$
\textrm{Prob}( a = 0 ) = \frac{1}{2} + \frac{1}{2}\vert \langle \psi \vert \phi \rangle \vert ^2 ,
$$

which we can use to estimate the value of $\vert \langle \psi \vert \phi \rangle \vert ^2 $ to arbitrary accuracy as the number of shots on the device increases.   

## Exercise: Calculate an inner product

Calculate the inner product of two states $\vert \phi \rangle$ and $\vert \psi \rangle$ of your own choosing using what you have learned in this notebook. 

Hints:

First you need to initialise the two states $\vert \phi \rangle$ and $\vert \psi \rangle$ using the single qubit rotations from the start of the notebook.

Then you need to apply the SWAP test and then do postprocessing of the results to calculate the inner product. 

Use the simulator to remove the need for error mitigation.

In [ ]:
### Your code here

In [ ]:
### Your code here

In [ ]:
### Your code here

[1] Adriano Barenco, André Berthiaume, David Deutsch, Artur Ekert, Richard Jozsa, and Chiara Macchiavello, Stabilization of Quantum Computations by Symmetrization, https://doi.org/10.1137/S0097539796302452

[2] Harry Buhrman, Richard Cleve, John Watrous, and Ronald de Wolf, Quantum Fingerprinting, 10.1103/PhysRevLett.87.167902

# PennyLane

PennyLane is another open source quantum computing language which is very useful for machine learning techniques. In this notebook we will go over the basics of building and executing a circuit in PennyLane.

Here we will use the PennyLane rotations to manipulate a single qubit state. The aim is to construct the state 

$$
\vert \psi \rangle = R_y(\theta) R_x (\phi) \vert 0 \rangle,
$$

where the rotations have the form

$$
R_x(\phi) = e^ {- i \phi \sigma_x/2} = \begin{pmatrix} \cos \frac{\phi}{2} & -i\sin \frac{\phi}{2} \\ -i\sin \frac{\phi}{2} & \cos \frac{\phi}{2} \end{pmatrix},  \hspace{5cm} R_y (\theta) = e^{-i \theta \sigma_y/2} = \begin{pmatrix} \cos \frac{\theta}{2} & -\sin \frac{\theta}{2} \\ \sin \frac{\theta}{2} & \cos \frac{\theta}{2} \end{pmatrix}.
$$

Unlike Qiskit, in PennyLane, you must always return a measurement operation. Therefore, in this example, we will measure the expectation value of the Pauli-Z operator, $\sigma_z$, such that 

$$
\langle \psi \vert \sigma_z \vert \psi \rangle = \langle 0 \vert R_x^\dagger (\phi) R_y^\dagger (\theta \sigma_z R_y(\theta) R_x (\phi) \vert 0 \rangle = \cos \phi \cos \theta.
$$

Let's now encode this onto a circuit using PennyLane.

In [1]:
import pennylane as qml
from pennylane import numpy as npl

When coding using PennyLane, it is always important to import the wrapped version of NumPy from Pennylane. To not conflict with the above, we import this version as `npl`. This wrapped version allows you to:

- combine quantum functions (evaluated on quantum hardware/simulators) and classical functions (provided by NumPy)
- allow PennyLane to automatically calculate gradients of both classical and quantum functions

## Creating a device

To start, we must first create a `device` object. This is analogous to loading a backend in Qiskit. For this example, we will use the qubit system with one wire, which is one qubit in the PennyLane language. 

In [2]:
dev1 = qml.device("default.qubit", wires=1)

## Constructing a QNode

Once we have created a device, we must now construct a quantum node ([QNode](https://docs.pennylane.ai/en/stable/code/api/pennylane.QNode.html#pennylane.QNode)). QNodes are an abstract encapsulation of a quantum function, described by a quantum circuit. QNodes are bound to a particular quantum device, which is used to evaluate expectation and variance values of this circuit.

In [3]:
@qml.qnode(dev1, interface="autograd")
def circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    return qml.expval(qml.PauliZ(0))

Here we have defined two things. Firstly, we have to construct the quantum function that is to be evaluated by the QNode. Here we build a simple circuit consisting of two rotations, $R_X$ followed by $R_Y$. Building circuits in PennyLane is analogous to Qiskit, however the the circuit function must return either a single or a tuple of measured observables. This reduces the flexibility in building circuits that is on offer from Qiskit, however PennyLane is very powerful when it comes to Quantum Machine Learning. 

Secondly, we must then convert the circuit into a QNode using the `qnode` function. This must be constructed directly above the quantum function definition. Here we define our `circuit` function as a QNode which will run on the `dev1` device each time it is evaluated. 

To evaluate, we just call the quantum function:

In [4]:
phi    = 0.34
theta  = 0.78
result = circuit([phi, theta]) 
print(result)

0.6702170547483379


## Calculating gradients

PennyLane allows for seemless integration of quantum circuits with classsical computing techniques. PennyLane incorporates both analytic differentiation, as well as numerical methods to calculate the gradients of a quantum circuit. This is done using the `grad` function

In [5]:
dcircuit = qml.grad(circuit, argnum=0)

where the `grad` function returns another QNode representing the gradient of the circuit. We can evaluate this QNode in the same way as above

In [7]:
gradient = dcircuit([phi, theta])
print(gradient)

[array(-0.23708049), array(-0.66301995)]


## Simple Optimisation

PennyLane has in-built optimisers which can be used within the PennyLane language to optimise quantum circuits. Here we use the circuit from above and optimise $\phi$ and $\theta$ to get a circuit which rotates the state $\vert 0 \rangle$ to the $\vert 1 \rangle$ state. In terms of our measurement of the Pauli-Z operator, this means rotating from the measurement of $1$ to $-1$. 

Here we can define our cost function simply as the output of the circuit, as the measurement of the expectation value of the Pauli-Z operator is bounded between [-1, 1]. 

In [8]:
def cost(x):
    return circuit(x)

We now define some initial values which give us a result close to 1, thus the $\vert 0 \rangle$ state. 

In [10]:
init_params = npl.array([0.011, 0.012], requires_grad=True)
print(cost(init_params))

0.9998675058299389


Note: here you must be careful not to use `np` and use `npl`. 

We now perform an optimisation to get create a circuit which rotates the qubit to the $\vert 1 \rangle$ state. 

In [11]:
### Initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4) # we use a good lr

steps = 100

### Set the initial parameter values
params = init_params

### Perform optimisation, updating the parameters at each step
for i in range(steps):
    params = opt.step(cost, params)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.9961778
Cost after step    10:  0.8974944
Cost after step    15:  0.1440490
Cost after step    20: -0.1536720
Cost after step    25: -0.9152496
Cost after step    30: -0.9994046
Cost after step    35: -0.9999964
Cost after step    40: -1.0000000
Cost after step    45: -1.0000000
Cost after step    50: -1.0000000
Cost after step    55: -1.0000000
Cost after step    60: -1.0000000
Cost after step    65: -1.0000000
Cost after step    70: -1.0000000
Cost after step    75: -1.0000000
Cost after step    80: -1.0000000
Cost after step    85: -1.0000000
Cost after step    90: -1.0000000
Cost after step    95: -1.0000000
Cost after step   100: -1.0000000
Optimized rotation angles: [7.15266381e-18 3.14159265e+00]


We see that, to get a state of $\vert 1 \rangle$ we must rotate the system by $R_X(0)$ and $R_Y(\pi)$, which makes sense as

$$
\langle \psi \vert \sigma_z \vert \psi \rangle = \cos \phi \cos \theta.
$$

## Exercise:

Here we used the fact that $-1$ is the optimal answer to our question, however what do we do if we wanted to achieve a state 

$$
\frac{1}{\sqrt{2}} \Big( \vert 0 \rangle + \vert 1 \rangle \Big).
$$

How would this change the cost function? Can you write up an optimisation to achive this state, sometimes called the $\vert + \rangle$ state. 